# Radiomics Feature Extraction

In this notebook we will show you how to extract features in batch mode using the command line interface (cli) approach. 

We will use data from https://github.com/rcuocolo/PROSTATEx_masks repository

First we need check and install the required libraries to more easily perform these steps.

In [75]:
!python -c "import gdown" || pip install --progress-bar off -q gdown
!python -c "import SimpleITK" || pip install --upgrade --pre SimpleITK --find-links https://github.com/SimpleITK/SimpleITK/releases/tag/latest --progress-bar off -q
!python -c "import matplotlib" || pip install --progress-bar off -q matplotlib
!python -c "import radiomics" || pip install --progress-bar off -q pyradiomics
!python -c "import yaml" || pip install --progress-bar off -q pyyaml
!python -c "import pandas" || pip install --progress-bar off -q pandas

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'yaml'
ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml


# Download ProstateX images and mask and parameter extraction file 

In [71]:
import os
if not os.path.isdir('input_t2w.csv'): 
    !gdown --id 1dNqdVsN0PlCG01sl6wUCh9QJ5NnkOrDS -q
    !unzip lesions_T2.zip >/dev/null
    !rm lesions_T2.zip >/dev/null
    !rm -r __MACOSX >/dev/null
else:
    print("lesions_T2 folder already exists, the zip was not downloaded")

if not os.path.isfile('Params_T2_BW.yaml'):
    !gdown --id 12VOQCPFK4HARrwQzbB4fr_f2PepGPFWm -q
else:
    print("Params_T2_BW.yaml already exists, the file was not downloaded")

print('Done!')

/Users/joaosantinha/miniconda3/envs/RadiomicsEssentials/lib/python3.8/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1dNqdVsN0PlCG01sl6wUCh9QJ5NnkOrDS
From (redirected): https://drive.google.com/uc?id=1dNqdVsN0PlCG01sl6wUCh9QJ5NnkOrDS&confirm=t&uuid=933c504b-89ba-4d49-a01f-fdfe3696b7d9
To: /Users/joaosantinha/CODE/RadiomicsEurRadEssentialsPaper/lesions_T2.zip
100%|████████████████████████████████████████| 767M/767M [02:16<00:00, 5.61MB/s]
/Users/joaosantinha/miniconda3/envs/RadiomicsEssentials/lib/python3.8/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=12VOQCPFK4HARrwQzbB4fr_f2PepG

## Import required libraries

In [76]:
#import yaml
import os
import pandas as pd
import csv

### Lets have a look at our parameter file

Select the folder icon on left bar and double click on Params_T2_BW.yaml file to see it.

#### Normalization
Since we are working T2w MRI images we will normalize the images by using the parameter `normalize` which we set as `true`. This set the mean intensity of the images at zero and scale the standard deviation to be equal to one. We then scale the standard deviation of the image intensities to be 100 by setting `normalizeScale` to `true`.

The computation of some features (namely Energy, Total Energy and RMS) is affected by the existance of negative value and since we have normalized our images to have mean 0 and standard deviation of 100 we need to ensure that the majority of our normalized image intensities is equal or above 0. We can do this using the `voxelArrayShift`. We selected a value of `300`, which assuming a normal distribution of the image intensities would ensure that 99.7% of them are above 0.

#### Memory savings
The `preCrop` set to `true` allows us to save some memory during feature computation (and possibly preventing errors).

#### Feature extraction type (2D vs 3D)
We have a dataset where our images are anisotropic (voxel dimensions are not the same for all three axes) and, more specifically, the slice thickness is much higher than the in plane resolution. Because of this, we will perform the extraction of 2D radiomic features instead of 3D. Since our images are T2w Axial images we need to define that we wish to perform a 2D feature extraction by setting `force2D` to `true` and set the in-plane axis to `0` using `force2Ddimension` (Axial - 0, Coronal - 1, Saggital - 2).

#### Interpolation/Resampling
To ensure that we are extracting radiomic feature that are comparable among different images we need to standardize the in-plane voxel dimensions. We do this by setting the `interpolator` to `sitkBSpline` and the `resampledPixelSpacing` to a chosen resolution `[0.6, 0.6, 0]` (the 0 on the third position means that we will not resample the slice thickness and only the in-plane resolution of the images will be transformed to 0.6 mm x 0.6 mm, which is ok because the features will be extracted in 2D).

#### Resegmentation
The resegmentation allows the mask to be redefine to, for example, discard intensities in our mask that are outside $mean\ \pm\ 3\times std$. We can do this by setting `resegmentMode` to `sigma` and `resegmentRange` to `[-3, 3]`.

#### Intensity discritization
To ensure that across images the number of bins used for the feature extraction we are between have across patients 30 to 130 performed a first feature extraction to get the `original_firstorder_Range` of our patients and we select the `binWidth` so that it is higher than 30 and lower than 130 accross all patients.

#### Filters and Features
We use the `imageType` to define the types of images that from which we which to extract the radiomic features and `featureClass` to select the type of features to be extracted.

# Perform Radiomic Feature Extraction - Create the input file

For the radiomic feature extraction we will use PyRadiomics batch mode, where we will have to provide as input `.csv` file contain 3 columns. The first column is the `ID`, the second the `Image` and the third the `Mask`, as shown below.

In [77]:
if not os.path.isfile('input_t2w.csv'):
    dir_image_path = 'lesions_T2/Images'
    dir_mask_path = 'lesions_T2/Masks'

    img_files = sorted([os.path.join(dir_image_path, x) for x in os.listdir(dir_image_path) if '.nii' in x])
    msk_files = sorted([os.path.join(dir_mask_path, x) for x in os.listdir(dir_mask_path) if '.nii.gz' in x])

    input_csv_t2 = [["ID", "Image", "Mask"]]
    for msk_file in msk_files:
        patient_id = os.path.basename(msk_file)[:14]
        image = [x for x in img_files if patient_id in x][0]
        input_csv_t2.append([patient_id, image, msk_file])

    with open('input_t2w.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(input_csv_t2)

### Check the Input File

In [78]:
input_df = pd.read_csv('input_t2w.csv')
input_df.head()

,ID,Image,Mask
0,ProstateX-0000,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...
1,ProstateX-0001,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...
2,ProstateX-0002,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...
3,ProstateX-0002,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...
4,ProstateX-0003,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...,/Users/joaosantinha/CODE/RadiomicsEurRadEssent...


# Required inputs to PyRadiomics batch execution

Besides the input file we will have to provide the filepath for the output file where the feature values for the different images will be saved (use `-o` before indicating the path), define that we want the output as a csv file by adding `-f csv`, and finally indicating the filepath of the parameter file using `-p`

Our command looks like this:

`pyradiomics input_t2w.csv -o output_t2w.csv -f csv -p Params_T2_BW.yaml`

# Execute Batch Radiomic Feature Extraction

Are advised to run the following line in your terminal or command line to speed up the extraction. To do so, remove the initial "!" in the following line

In [82]:
!pyradiomics input_t2w.csv -o output_t2w.csv -f csv -p Params_T2_BW.yaml


[2024-05-08 08:09:18] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2024-05-08 08:09:19] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2024-05-08 08:09:19] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2024-05-08 08:09:19] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2024-05-08 08:09:20] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2024-05-08 08:09:21] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2024-05-08 08:09:21] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2024-05-08 08:09:22] W: radiomics.glcm: GLCM is symmet

# Lets check the radiomics output file

As you can see it containts the same three columns (ID, Image, Mask) of our input, tnen collumns providing information about the versions, image properties, and feature extraction settings, followed by the values of the extracted radiommic features.


In [ ]:
radiomics_df = pd.read_csv('output_t2w.csv')
radiomics_df.head()

,ID,Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,diagnostics_Image-interpolated_Spacing,diagnostics_Image-interpolated_Size,diagnostics_Image-interpolated_Mean,diagnostics_Image-interpolated_Minimum,diagnostics_Image-interpolated_Maximum,diagnostics_Mask-interpolated_Spacing,diagnostics_Mask-interpolated_Size,diagnostics_Mask-interpolated_BoundingBox,diagnostics_Mask-interpolated_VoxelNum,diagnostics_Mask-interpolated_VolumeNum,diagnostics_Mask-interpolated_CenterOfMassIndex,diagnostics_Mask-interpolated_CenterOfMass,diagnostics_Mask-interpolated_Mean,diagnostics_Mask-interpolated_Minimum,diagnostics_Mask-interpolated_Maximum,...,exponential_glrlm_RunPercentage,exponential_glrlm_RunVariance,exponential_glrlm_ShortRunEmphasis,exponential_glrlm_ShortRunHighGrayLevelEmphasis,exponential_glrlm_ShortRunLowGrayLevelEmphasis,exponential_glszm_GrayLevelNonUniformity,exponential_glszm_GrayLevelNonUniformityNormalized,exponential_glszm_GrayLevelVariance,exponential_glszm_HighGrayLevelZoneEmphasis,exponential_glszm_LargeAreaEmphasis,exponential_glszm_LargeAreaHighGrayLevelEmphasis,exponential_glszm_LargeAreaLowGrayLevelEmphasis,exponential_glszm_LowGrayLevelZoneEmphasis,exponential_glszm_SizeZoneNonUniformity,exponential_glszm_SizeZoneNonUniformityNormalized,exponential_glszm_SmallAreaEmphasis,exponential_glszm_SmallAreaHighGrayLevelEmphasis,exponential_glszm_SmallAreaLowGrayLevelEmphasis,exponential_glszm_ZoneEntropy,exponential_glszm_ZonePercentage,exponential_glszm_ZoneVariance,exponential_ngtdm_Busyness,exponential_ngtdm_Coarseness,exponential_ngtdm_Complexity,exponential_ngtdm_Contrast,exponential_ngtdm_Strength,exponential_gldm_DependenceEntropy,exponential_gldm_DependenceNonUniformity,exponential_gldm_DependenceNonUniformityNormalized,exponential_gldm_DependenceVariance,exponential_gldm_GrayLevelNonUniformity,exponential_gldm_GrayLevelVariance,exponential_gldm_HighGrayLevelEmphasis,exponential_gldm_LargeDependenceEmphasis,exponential_gldm_LargeDependenceHighGrayLevelEmphasis,exponential_gldm_LargeDependenceLowGrayLevelEmphasis,exponential_gldm_LowGrayLevelEmphasis,exponential_gldm_SmallDependenceEmphasis,exponential_gldm_SmallDependenceHighGrayLevelEmphasis,exponential_gldm_SmallDependenceLowGrayLevelEmphasis
0,Prostatex0014,/content/ProstateXSegmentationsExample/Prostat...,/content/ProstateXSegmentationsExample/Prostat...,v3.0.1,1.19.5,2.0.2,1.1.1,3.7.10,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3, 5]}, 'Wa...",c16babc0c68c81ac92970854b980dfee4a5f136c,3D,"(0.5, 0.5, 3.0000007152557373)","(384, 384, 19)",224.535214,0.0,1065.0,06ef90b85db04b9ecf100467e794c0354986e829,"(0.5, 0.5, 3.0000007152557373)","(384, 384, 19)","(149, 166, 4, 91, 68, 15)",35003,1,"(193.09330628803247, 198.6340885066994, 9.5433...","(-16.62551940263117, 35.68184289389275, 12.771...","(0.6, 0.6, 3.0000007152557373)","(88, 69, 19)",6.478077,-148.098153,329.303403,"(0.6, 0.6, 3.0000007152557373)","(88, 69, 19)","(6, 6, 4, 76, 57, 15)",24334,1,"(42.72158297032958, 33.35341497493219, 9.54154...","(-16.689222764449624, 35.62657559464135, 12.77...",-14.609330,-145.098995,329.303403,...,0.044182,287.613363,0.121375,0.335531,0.067836,53.481013,0.676975,0.161513,3.392405,6.985922e+05,6.985993e+05,6.985904e+05,0.40